RNA-seq quantification:

```
$ gsutil cp gs://musc-tcr-transfer/output/openvax/201912-mc38b/rna_L001_R1.fastq.gz .
$ gsutil cp gs://musc-tcr-transfer/output/openvax/201912-mc38b/rna_L001_R2.fastq.gz .

$ wget "http://ftp.ensembl.org/pub/release-103/fasta/mus_musculus/cdna/Mus_musculus.GRCm39.cdna.all.fa.gz"
$ kallisto index -i Mus_musculus.GRCm39.cdna.all.fa.gz Mus_musculus.GRCm39.cdna.all.fa.gz

$ kallisto quant -o MC38B-kallisto -i Mus_musculus.GRCm39.cdna.all.fa.gz rna_L001_R1.fastq.gz rna_L001_R2.fastq.gz
```

Binding prediction:

```
$ conda create -n mhc
$ conda activate mhc
$ pip install mhcflurry mhctools topiary
$ pyensembl install --release 102 --species mus_musculus

$ gsutil cp gs://musc-tcr-transfer/output/openvax/201912-mc38b/mutect.vcf .
$ gsutil cp gs://musc-tcr-transfer/output/openvax/201912-mc38b/strelka.vcf .

$ topiary --mhc-predictor mhcflurry --mhc-alleles "H-2-Kb H-2Db" --vcf mutect.vcf --vcf strelka.vcf --output-csv MC38B-neoepitopes.csv --output-html MC38B-neoepitopes.html
```

In [1]:
library('tidyverse')

── Attaching packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
epitopes <-
    read_csv('MC38B-neoepitopes.csv')

epitopes %>% head


── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  `#` = col_double(),
  variant = col_character(),
  peptide_offset = col_double(),
  peptide = col_character(),
  allele = col_character(),
  score = col_double(),
  affinity = col_double(),
  percentile_rank = col_double(),
  prediction_method_name = col_character(),
  peptide_length = col_double(),
  gene = col_character(),
  gene_id = col_character(),
  transcript_id = col_character(),
  transcript_name = col_character(),
  effect = col_character(),
  effect_type = col_character(),
  contains_mutant_residues = col_logical(),
  mutation_start_in_peptide = col_double(),
  mutation_end_in_peptide = col_double()
)




#,variant,peptide_offset,peptide,allele,score,affinity,percentile_rank,prediction_method_name,peptide_length,gene,gene_id,transcript_id,transcript_name,effect,effect_type,contains_mutant_residues,mutation_start_in_peptide,mutation_end_in_peptide
<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<dbl>,<dbl>
0,chr5 g.66022672C>T,491,AAAAAAAAT,H-2-Kb,0.1496558,9902.4301,8.350875,mhcflurry,9,Rbm47,ENSMUSG00000070780,ENSMUST00000201544,Rbm47-209,p.A500T,Substitution,TRUE,8,9
1,chr5 g.66022672C>T,492,AAAAAAATA,H-2-Kb,0.2172512,4765.5511,5.062875,mhcflurry,9,Rbm47,ENSMUSG00000070780,ENSMUST00000201544,Rbm47-209,p.A500T,Substitution,TRUE,7,8
2,chr6 g.125101598G>T,219,AAAAAAAVS,H-2-Kb,0.1190600,13788.2909,11.425500,mhcflurry,9,Chd4,ENSMUSG00000063870,ENSMUST00000056889,Chd4-201,p.A228S,Substitution,TRUE,8,9
3,chr2 g.27108424_27108425insC,958,AAAAAAPEI,H-2-Kb,0.3400278,1262.3789,2.355375,mhcflurry,9,Adamtsl2,ENSMUSG00000036040,ENSMUST00000091233,Adamtsl2-201,p.H956fs,FrameShift,TRUE,0,9
4,chr5 g.20702109C>A,1234,AAAAAAQGL,H-2-Kb,0.3734419,879.3819,1.897000,mhcflurry,9,Magi2,ENSMUSG00000040003,ENSMUST00000197354,Magi2-205,p.P1241Q,Substitution,TRUE,6,7
5,chr5 g.139544154C>A,45,AAAAAASVH,H-2-Kb,0.1057919,15916.8408,13.541250,mhcflurry,9,Uncx,ENSMUSG00000029546,ENSMUST00000172997,Uncx-201,p.P54H,Substitution,TRUE,8,9


In [3]:
exp_tbl <-
    read_tsv('./MC38B-kallisto/abundance.tsv') %>%
    separate(`target_id`, c('transcript_id', 't_version'), sep='\\.')

exp_tbl %>% head


── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  target_id = col_character(),
  length = col_double(),
  eff_length = col_double(),
  est_counts = col_double(),
  tpm = col_double()
)




transcript_id,t_version,length,eff_length,est_counts,tpm
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
ENSMUST00000178537,2,12,7.20,0,0
ENSMUST00000178862,2,14,9.20,0,0
ENSMUST00000196221,2,9,5.75,0,0
ENSMUST00000179664,2,11,7.75,0,0
ENSMUST00000177564,2,16,11.20,0,0
ENSMUST00000179520,2,11,7.75,0,0


In [4]:
neoantigens_tbl <-
    epitopes %>%
    left_join(
        exp_tbl %>% select(`transcript_id`, `est_counts`, `TPM`=`tpm`), 
        by='transcript_id'
    ) %>%
    arrange(`affinity`) %>%
    select(-`#`) %>%
    relocate(`gene`, `peptide`, `allele`, `affinity`, `TPM`)
    

neoantigens_tbl

gene,peptide,allele,affinity,TPM,variant,peptide_offset,score,percentile_rank,prediction_method_name,peptide_length,gene_id,transcript_id,transcript_name,effect,effect_type,contains_mutant_residues,mutation_start_in_peptide,mutation_end_in_peptide,est_counts
<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<dbl>,<dbl>,<dbl>
Olfr1477,VSFTFHLSL,H-2-Kb,15.82296,0.00000e+00,chr19 g.13502843T>C,158,0.7447764,0.002875,mhcflurry,9,ENSMUSG00000071629,ENSMUST00000217001,Olfr1477-202,p.F167L,Substitution,TRUE,8,9,0.00000e+00
Slit1,SSFKHMPKL,H-2-Kb,16.88022,0.00000e+00,chr19 g.41651178G>T,198,0.7387985,0.003500,mhcflurry,9,ENSMUSG00000025020,ENSMUST00000025993,Slit1-201,p.N202K,Substitution,TRUE,3,4,0.00000e+00
Vmn1r88,YTFVFKPHL,H-2-Kb,19.93099,1.15871e+00,chr7 g.13177821T>G,31,0.7234439,0.011250,mhcflurry,9,ENSMUSG00000117853,ENSMUST00000236472,Vmn1r88-202,p.F35V,Substitution,TRUE,3,4,4.34789e+02
Aldh1l1,SSYPWRSSM,H-2-Kb,19.93408,0.00000e+00,chr6 g.90557285_90557285delA,87,0.7234296,0.011250,mhcflurry,9,ENSMUSG00000030088,ENSMUST00000130418,Aldh1l1-203,p.K21fs,FrameShift,TRUE,0,9,0.00000e+00
Adpgk,ASMTNMELM,H-2-Db,20.89715,1.73820e+01,chr9 g.59313823G>T,298,0.7190688,0.000500,mhcflurry,9,ENSMUSG00000025236,ENSMUST00000217570,Adpgk-205,p.R304M,Substitution,TRUE,5,6,1.62606e+03
Gabrg1,KTFLFSPSL,H-2-Kb,20.95931,0.00000e+00,chr5 g.70842204C>T,4,0.7187944,0.013375,mhcflurry,9,ENSMUSG00000001260,ENSMUST00000031119,Gabrg1-201,p.A6T,Substitution,TRUE,1,2,0.00000e+00
Olfr1442,ISFIYFNNV,H-2-Kb,21.28552,0.00000e+00,chr19 g.12674310C>A,28,0.7173669,0.014625,mhcflurry,9,ENSMUSG00000044441,ENSMUST00000208494,Olfr1442-203,p.T35N,Substitution,TRUE,6,7,0.00000e+00
Hdgfl2,KGYPHWPAL,H-2-Kb,21.38661,6.05999e+00,chr17 g.56079825G>T,15,0.7169290,0.014625,mhcflurry,9,ENSMUSG00000002833,ENSMUST00000225843,Hdgfl2-210,p.R24L,Substitution,TRUE,8,9,5.03605e+02
Zbtb40,KSFHFYCPL,H-2-Kb,22.34975,5.95220e+00,chr4 g.136995463C>G,760,0.7128578,0.017375,mhcflurry,9,ENSMUSG00000060862,ENSMUST00000049583,Zbtb40-201,p.R768P,Substitution,TRUE,7,8,1.74671e+03


In [5]:
neoantigens_tbl %>%
    write_csv('MC38B-neoantigens_with_expression.csv')